# **警告：該資料庫和工具皆為本人所製，請勿任意下載。**F113119134 高雄科技大學

In [ ]:
!git clone https://github.com/113119134HAUNG/Weather-AI-Agent.git> /dev/null 2>&1 && \
!!bash install.sh > /dev/null 2>&1 && \
echo "全部安裝完畢!"

全部安裝完畢!


# **1. 讀入套件、自製工具和資料集**

**這裡主要用 `OpenHowNet` 匯入語義(詞)資料庫,opencc-因為資料庫是簡字,所以需要繁轉簡，輸出可以再簡轉繁。**

In [ ]:
%matplotlib inline

import re
import os
import json
import faiss
import jieba
import torch
import openai
import requests
import importlib
import OpenHowNet
import numpy as np
import pandas as pd
import gradio as gr
import sememe_tools as st
import matplotlib.pyplot as pltm
import vector_utils_advanced as vu
importlib.reload(vu)

from tqdm import tqdm
from pprint import pprint
from opencc import OpenCC
from datetime import datetime
from google.colab import userdata
from OpenHowNet import HowNetDict
from datasets import load_dataset
from collections import defaultdict
from multi_turn_qa import multi_turn_qa
from sklearn.preprocessing import normalize
from transformers import AutoTokenizer, AutoModel
from anytree import Node, RenderTree, AsciiStyle

vector_utils_advanced 模組載入完成！


In [ ]:
# 讀取 NLPCC-MH 資料集
with open("/content/NLPCC-MH/data/nlpcc-mh.train.json", "r", encoding="utf-8") as f:
    data = json.load(f)
print(data[0])  # 確認第一筆內容

{'q': '柳江的主要支流的发源地是哪里？', 'path': [['柳江 ||| 2970218', '主要支流', '寨蒿河 ||| 7476446'], ['寨蒿河 ||| 7476446', '发源于', '贵州省黎平县高洋乡 ||| 0']]}


In [ ]:
# 讀取自製 Synonyms 資料
with open("/content/sememe_synonym_OK.json", "r", encoding="utf-8") as f:
    custom_synonyms = json.load(f)

# 建立 custom_synonym_map, location_terms, weather_terms
custom_synonym_map = {}
location_terms = set()
weather_terms = set()

for key, entry in custom_synonyms.items():
    zh_entry = entry.get("zh", key)
    zh_words = zh_entry if isinstance(zh_entry, list) else [zh_entry]

    # 選第一個作為標準化詞
    standard_word = st.normalize_text(zh_words[0])

    # 主詞 + 同義詞都標準化
    for word in zh_words:
        if isinstance(word, str):
            custom_synonym_map[st.normalize_text(word)] = standard_word
    for syn in entry.get("synonyms", []):
        if isinstance(syn, str):
            custom_synonym_map[st.normalize_text(syn)] = standard_word

    # 分類地名與天氣詞
    categories = entry.get("categories", {})
    target_set = location_terms if any(c in categories for c in ["直轄市", "省轄市", "縣", "縣轄市", "區", "鎮", "鄉"]) else weather_terms
    for w in zh_words:
        if isinstance(w, str):
            target_set.add(st.normalize_text(w))

In [ ]:
import json
import re
from collections import defaultdict

# 扁平化資料結構
def flatten_sememe_data(data, path=None, results=None):
    if results is None:
        results = {}
    if path is None:
        path = []
    if isinstance(data, dict):
        if "items" in data:
            for item in data["items"]:
                key = item.get("id") or item.get("zh") or item.get("en")
                if not key:
                    continue
                linked = item.get("linked_sememe", {})
                synonyms = []
                if isinstance(linked, dict):
                    zh_syns = linked.get("zh", [])
                    en_syns = linked.get("en", [])
                    zh_syns = zh_syns if isinstance(zh_syns, list) else [zh_syns]
                    en_syns = en_syns if isinstance(en_syns, list) else [en_syns]
                    synonyms = list(set(filter(None, zh_syns + en_syns)))
                results[key] = {
                    "zh": item.get("zh", ""),
                    "en": item.get("en", ""),
                    "synonyms": synonyms,
                    "categories": path.copy()
                }
        if "categories" in data:
            for cat_name, cat_data in data["categories"].items():
                flatten_sememe_data(cat_data, path + [cat_name], results)
    return results

# 簡易文字正規化
class SimpleNormalizer:
    @staticmethod
    def normalize_text(text):
        return re.sub(r"\s+", "", text.lower())

st = SimpleNormalizer()

# 類別定義與優先順序
CATEGORY_TREE = {
    "geo_feature": ["地形地貌", "水文環境", "火山與地質", "海洋與沿岸", "國家公園與自然保護區", "特殊自然景觀"],
    "climate": ["氣候", "氣候變遷", "季風", "乾季", "濕季", "氣候災害"],
    "weather": ["天氣", "晴朗與雲量變化", "降水與雷雨現象", "特殊降水與冰雪現象", "能見度與空氣現象", "極端天氣與災害", "鋒面與氣候變化", "天氣現象"],
    "location": ["直轄市", "省轄市", "縣", "縣轄市", "區", "鄉", "鎮", "城市", "都市區", "村", "里", "行政區"]
}
CATEGORY_PRIORITY = ["geo_feature", "climate", "weather", "location"]

# 語意矯正設定
WEATHER_OVERRIDE = ["冷鋒", "暖鋒", "滯留鋒", "鋒面雨", "雷陣雨", "短時強降雨", "間歇性小雨", "霜凍", "揚沙", "晴朗無雲"]
CLIMATE_EXCLUDE_FROM_WEATHER = ["強降雨事件", "年降雨量", "梅雨季"]

# 精準分類 + 語意補正 + fallback 城市判斷
def build_precise_maps(flattened_data):
    category_term_sets = {cat: set() for cat in CATEGORY_TREE.keys()}
    custom_synonym_map = {}
    classified_terms = set()
    unclassified_terms = set()
    reclassified_terms = []

    for key, entry in flattened_data.items():
        zh_entry = entry.get("zh", key)
        synonyms = entry.get("synonyms", [])
        zh_words = zh_entry if isinstance(zh_entry, list) else [zh_entry]
        standard_word = st.normalize_text(zh_words[0]) if zh_words else None
        if not standard_word:
            continue

        for word in zh_words + synonyms:
            if isinstance(word, str):
                custom_synonym_map[st.normalize_text(word)] = standard_word

        path = entry.get("categories", [])
        entry["classification"] = []
        entry["triggered_by"] = []

        classified = False
        for cat in CATEGORY_PRIORITY:
            keywords = CATEGORY_TREE[cat]
            matched = [p for p in path if any(k in p for k in keywords)]
            if matched:
                entry["classification"].append(cat)
                entry["triggered_by"].extend(matched)
                category_term_sets[cat].add(standard_word)
                classified_terms.add(standard_word)
                classified = True
                break

        # fallback 城市詞尾判斷
        if not classified:
            location_suffixes = ["市", "區", "鄉", "鎮", "村", "里", "島"]
            if any(isinstance(w, str) and w.endswith(tuple(location_suffixes)) for w in zh_words):
                category_term_sets["location"].add(standard_word)
                entry["classification"].append("location")
                entry["triggered_by"].append("suffix_match")
                classified_terms.add(standard_word)
                classified = True

        if not classified:
            unclassified_terms.add(standard_word)

    # 語意矯正 weather 與 climate 的誤歸類
    for word in list(classified_terms):
        original = None
        for cat, terms in category_term_sets.items():
            if word in terms:
                original = cat
                break

        if any(keyword in word for keyword in WEATHER_OVERRIDE):
            if word not in CLIMATE_EXCLUDE_FROM_WEATHER and original != "weather":
                if original:
                    category_term_sets[original].remove(word)
                category_term_sets["weather"].add(word)
                reclassified_terms.append((word, original, "weather"))

    return custom_synonym_map, category_term_sets, classified_terms, unclassified_terms, reclassified_terms

# 主程式
with open("/content/sememe_synonym_OK.json", "r", encoding="utf-8") as f:
    raw_data = json.load(f)

taiwan_data = raw_data.get("Country", {}).get("categories", {}).get("Taiwan", {})
flattened_data = flatten_sememe_data(taiwan_data)

custom_synonym_map, category_term_sets, classified_terms, unclassified_terms, reclassified_terms = build_precise_maps(flattened_data)

# 結果輸出
print(f"\n自訂 Synonym Map 已載入，共 {len(custom_synonym_map)} 筆\n")
for cat, terms in category_term_sets.items():
    print(f"分類「{cat}」詞彙數量：{len(terms)}")
    print(f"範例：{list(terms)[:10]}\n")

total_classified = sum(len(terms) for terms in category_term_sets.values())
print(f"已分類詞彙總數：{total_classified}")
print(f"未分類詞彙總數：{len(unclassified_terms)}")
if unclassified_terms:
    print(f"未分類範例：{list(unclassified_terms)[:10]}")
if reclassified_terms:
    print("\n語意矯正重新分類：")
    for word, from_cat, to_cat in reclassified_terms:
        print(f"    {word}：{from_cat} → {to_cat}")


✅ 自訂 Synonym Map 已載入，共 919 筆

✅ 分類「geo_feature」詞彙數量：49
範例：['墾丁國家公園', '中央山脈', '花蓮溪', '北海岸', '屏東平原', '合歡山', '曾文水庫', '南海岸', '大雪山', '墾丁珊瑚礁']

✅ 分類「climate」詞彙數量：30
範例：['山區氣候差異', '氣候災害應對', '乾季', '南部乾旱區', '東北季風影響', '熱帶氣候', '海岸氣候特徵', '亞熱帶氣候', '年降雨量', '微氣候分布']

✅ 分類「weather」詞彙數量：40
範例：['間歇性小雨', '冷鋒通過', '短時強降雨', '強雷陣雨', '大雨', '揚沙', '霜凍', '晴朗無雲', '暖鋒通過', '局部驟雨']

✅ 分類「location」詞彙數量：322
範例：['西港區', '彰化市', '義竹鄉', '光復鄉', '大埔鄉', '莿桐鄉', '臺西鄉', '綠島鄉', '西湖鄉', '樹林區']

✅ 已分類詞彙總數：441
✅ 未分類詞彙總數：0

✅ 語意矯正重新分類：
    暖鋒通過：climate → weather
    鋒面雨：climate → weather
    冷鋒通過：climate → weather
    滯留鋒：climate → weather


In [ ]:
# 設定 custom_synonym_map 和 custom_synonyms 給 sememe_tools
st.set_custom_synonym_map(custom_synonym_map)
st.set_custom_synonyms(custom_synonyms)

自訂 Synonym Map 已載入，共 3 筆
自訂 Synonyms 擴展資料已載入，共 1 筆


In [ ]:
# 處理 NLPCC-MH 單筆問答樣本
def process_nlpccmh_sample(sample, base_id="nlpcc", index=0):
    result = []
    question = sample["q"]

    sememe_analysis = st.analyze_sentence(question)
    sememe_tags = sememe_analysis["sememe_tags"]
    sememe_map = sememe_analysis["sememe_map"]

    for path_id, triple in enumerate(sample.get("path", [])):
        head = triple[0].split(" ||| ")[0]
        relation = triple[1]
        tail = triple[2].split(" ||| ")[0]
        sentence = f"{head} {relation} {tail}"

        result.append({
            "id": f"{base_id}_{index}_{path_id}",
            "question": question,
            "question_sememe": sememe_tags,
            "question_sememe_map": sememe_map,
            "triple_sentence": sentence,
            "head": head,
            "relation": relation,
            "tail": tail
        })

    return result

In [ ]:
# 分批處理整份資料集並輸出為 JSONL
def process_nlpccmh_file(input_path, output_path, batch_size=1000):
    with open(input_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    with open(output_path, "w", encoding="utf-8") as out_f:
        for i in range(0, len(data), batch_size):
            batch = data[i:i + batch_size]
            for j, sample in enumerate(tqdm(batch, desc=f"Batch {i // batch_size}")):
                processed = process_nlpccmh_sample(sample, index=i + j)
                for entry in processed:
                    out_f.write(json.dumps(entry, ensure_ascii=False) + "\n")

In [ ]:
# 啟動處理流程
process_nlpccmh_file(
    input_path="/content/NLPCC-MH/data/nlpcc-mh.train.json",
    output_path="/content/NLPCC-MH/data/nlpcc-mh.train_sememe.jsonl"
)

**測試**

In [ ]:
# 測試資料
sememe_map = {
    "發源地": ["place|地方", "ComeToWorld|問世"],
    "柳江": ["waters|水域", "China|中國"]
}
question = "柳江的主要支流的发源地是哪里？"

# 從 sememe_tool.py 調用 generate_augmented_query
augmented_query = st.generate_augmented_query(question, sememe_map)

# 顯示結果
print(augmented_query)

**執行嵌入和建立向量庫**

In [ ]:
# 設定與模型初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-base-zh")
hf_model = AutoModel.from_pretrained("BAAI/bge-base-zh").to(device).eval()

In [ ]:
#構建 NLPCC-MH 擴充後的向量庫
def prepare_nlpccmh_augmented_data(
    input_path, index_path, meta_path, model, tokenizer, device, pooling="cls"
):
    assert os.path.exists(input_path), f"找不到輸入檔案: {input_path}"

    texts, ids, metas = [], [], []
    with open(input_path, "r", encoding="utf-8") as f:
        for i, line in enumerate(tqdm(f, desc="處理 NLPCC-MH 資料")):
            entry = json.loads(line.strip())
            question = entry["question"]
            sememe_map = entry["question_sememe_map"]

            pseudo_text = "；".join(st.format_sememe_map(sememe_map, style="display"))
            merged_query = st.generate_augmented_query(question, sememe_map)

            texts.append(merged_query)
            ids.append(entry["id"])
            metas.append({
                "query": question,
                "sememe": pseudo_text
            })

    vu.build_faiss_index_and_save(
        texts=texts,
        ids=ids,
        meta_list=metas,
        model=model,
        tokenizer=tokenizer,
        device=device,
        index_path=index_path,
        meta_path=meta_path,
        pooling=pooling
    )

    print(f"NLPCC-MH向量庫已建置完成，共 {len(texts)} 筆資料。")

In [ ]:
# 處理自製同義詞資料，建立向量庫
def prepare_custom_augmented_data(synonym_path, index_path, meta_path, model, tokenizer, device, pooling="cls"):
    assert os.path.exists(synonym_path), f"找不到輸入檔案: {synonym_path}"

    texts, ids, metas = [], [], []
    with open(synonym_path, "r", encoding="utf-8") as f:
        synonym_data = json.load(f)

    for i, (key, entry) in enumerate(tqdm(synonym_data.items(), desc="處理自製同義詞資料")):
        zh_entry = entry.get("zh", key)
        synonyms = entry.get("synonyms", [])
        categories = entry.get("categories", {})

        if isinstance(zh_entry, list):
            standard_word = zh_entry[0]
            parts = zh_entry.copy()
        else:
            standard_word = zh_entry
            parts = [zh_entry]

        parts.extend(synonyms)

        description = "、".join(parts) + "。這些是相關語義擴展資訊。"
        merged = f"[Q] {standard_word} [SEP] {description}"

        texts.append(merged)
        ids.append(f"custom_{i}")
        metas.append({
            "term": standard_word,
            "synonyms": synonyms,
            "categories": categories,
            "is_location": False
        })

        for cat_name, city_list in categories.items():
            for city in city_list:
                texts.append(f"[Q] {city} [SEP] {standard_word}地區")
                ids.append(f"city_{i}_{city}")
                metas.append({
                    "term": city,
                    "synonyms": [],
                    "categories": {},
                    "is_location": True
                })

    vu.build_faiss_index_and_save(
        texts=texts,
        ids=ids,
        meta_list=metas,
        model=model,
        tokenizer=tokenizer,
        device=device,
        index_path=index_path,
        meta_path=meta_path,
        pooling=pooling
    )
    print(f"自製 Synonym 向量庫已建置完成，共 {len(texts)} 筆資料。")

In [ ]:
# 建立 NLPCC-MH 與自製 Synonyms 向量庫
def run_all_indexing(nlpcc_input_path,nlpcc_index_path,nlpcc_meta_path,synonym_path,custom_index_path,custom_meta_path,model,tokenizer,device):
    print("開始全流程向量庫建立...")

    # 先建立 NLPCC
    print("\n開始建立 NLPCC-MH 向量庫...")
    prepare_nlpccmh_augmented_data(
        input_path=nlpcc_input_path,
        index_path=nlpcc_index_path,
        meta_path=nlpcc_meta_path,
        model=model,
        tokenizer=tokenizer,
        device=device)
    # 建立 Custom
    print("\n開始建立自製 Synonym 向量庫...")
    prepare_custom_augmented_data(
        synonym_path=synonym_path,
        index_path=custom_index_path,
        meta_path=custom_meta_path,
        model=model,
        tokenizer=tokenizer,
        device=device)
    print("\n全部向量庫建立完成！")
# 執行建立向量資料庫
run_all_indexing(nlpcc_input_path="/content/NLPCC-MH/data/nlpcc-mh.train_sememe.jsonl",nlpcc_index_path="/content/index.faiss",nlpcc_meta_path="/content/metadata.jsonl",synonym_path="/content/sememe_synonym.json",custom_index_path="/content/custom_index.faiss",custom_meta_path="/content/custom_metadata.jsonl",model=hf_model,tokenizer=tokenizer,device=device)

**查詢測試**

In [ ]:
# 載入所有索引
index_paths = [("/content/index.faiss", "/content/metadata.jsonl"),("/content/custom_index.faiss", "/content/custom_metadata.jsonl")]

indices, metadatas = [], []
for idx_path, meta_path in index_paths:
    index, meta = vu.load_index_and_metadata(idx_path, meta_path)
    indices.append(index)
    metadatas.append(meta)

print(f"成功載入 {len(indices)} 個索引庫！")

# 查詢
query = "[Q] 柳江的主要支流的发源地是哪里？ [SEP] 發源地 含有語意「place|地方、ComeToWorld|問世」；柳江 含有語意「waters|水域、China|中國」"

# 單一庫查詢
single_results = vu.search_with_metadata(
    query=query,
    index=indices[0],
    metadata=metadatas[0],
    model=hf_model,
    tokenizer=tokenizer,
    device=device,
    topk=3
)

print("\n單一庫查詢結果：")
for r in single_results:
    print(f"匹配 ID: {r['id']}｜分數: {r['score']:.4f}")
    print(f"原始問句/詞: {r['meta'].get('query', r['meta'].get('term', '')).strip()}")
    print(f"語義描述: {r['meta'].get('sememe', r['meta'].get('synonyms', ''))}\n")

# 多庫整合查詢
combined_results = vu.combine_search(
    query=query,
    indices=indices,
    metadatas=metadatas,
    model=hf_model,
    tokenizer=tokenizer,
    device=device,
    topk=5
)

print("多庫整合查詢結果：")
for r in combined_results:
    print(f"來自：{r['source']}｜匹配 ID: {r['id']}｜分數: {r['score']:.4f}")
    print(f"原始問句/詞: {r['meta'].get('query', r['meta'].get('term', '')).strip()}")
    print(f"語義描述: {r['meta'].get('sememe', r['meta'].get('synonyms', ''))}\n")

In [ ]:
# 將檢索結果整理為 RAG 輸入格式。
def build_rag_prompt(query, retrieved_results, mode="standard"):
    context_parts = []
    for r in retrieved_results:
        if mode == "standard":
            context_parts.append(
                f"- 問句：{r['meta'].get('query', r['text'])}\n  語義：{r['meta'].get('sememe', '')}"
            )
        elif mode == "flat":
            context_parts.append(r['text'])

    context_block = "\n\n".join(context_parts)
    prompt = f"你是一個中文問答助理，請根據下列檢索到的語意資訊，回答原始問題：\n\n{context_block}\n\n🔸 原始問題：{query}\n\n✅ 請用自然語言回答："
    return prompt

In [ ]:
# 取得 API 金鑰與 headers
api_key = userdata.get("Groq")
headers = {
    "Authorization": f"Bearer {api_key}",
    "Content-Type": "application/json"
}

# 取得模型列表
r = requests.get("https://api.groq.com/openai/v1/models", headers=headers)
data = r.json()
groq_model_id = data.get("data", [])
df = pd.DataFrame(groq_model_id)
if "id" in df.columns:
    df = df[["id"]]
    df.reset_index(drop=True, inplace=True)
    df.index += 1
    display(df)

# 指定使用模型
groq_model_id ="meta-llama/llama-4-maverick-17b-128e-instruct"
base_url = "https://api.groq.com/openai/v1"
os.environ["OPENAI_API_KEY"] = api_key

In [ ]:
# 載入索引
index1, meta1 = vu.load_index_and_metadata(index_path="/content/index.faiss",meta_path="/content/metadata.jsonl")
index2, meta2 = vu.load_index_and_metadata(index_path="/content/custom_index.faiss",meta_path="/content/custom_metadata.jsonl")

# 查詢問題
query = "柳江的主要支流的發源地是？"

# 用向量庫檢索（單輪查詢）
search_results = vu.search_with_metadata(
    query=query,
    index=index1,
    metadata=meta1,
    model=hf_model,
    tokenizer=tokenizer,
    device=device,
    topk=3
)

print("單輪檢索結果：")
for r in search_results:
    print(f"- 匹配分數: {r['score']:.4f} ｜內容: {r['text'][:50]}...")

# 多回合 QA（使用兩個索引）
history = multi_turn_qa(initial_query=query,index_list=[(index1, meta1), (index2, meta2)],model=hf_model,tokenizer=tokenizer,device=device,api_key=api_key,max_turns=3,topk=5)

# 顯示整個對話紀錄
print("\n多回合問答紀錄：")
for record in history:
    print(f"\n問題：{record['query']}")
    print(f"回答：{record['answer']}")

In [ ]:
# 載入 API 金鑰
api_keyW = userdata.get("天氣")

# 繁簡轉換
cc_sp2tw = OpenCC('s2twp')

# 文字正規化
def normalize_text(text):
    return cc_sp2tw.convert(text).replace("台", "臺").lower().strip()

# 問句預處理
def preprocess_query(query):
    query = normalize_text(query)
    useless_words = ["請問", "今天", "現在", "呢", "的", "請", "一下", "目前", "嗎", "呢？", "？"]
    for word in useless_words:
        query = query.replace(word, "")
    return query

In [ ]:
# 繁簡轉換 + 清洗
cc_sp2tw = OpenCC('s2twp')

def normalize_text(text):
    return cc_sp2tw.convert(text).replace("台", "臺").lower().strip()

def preprocess_query(query):
    query = cc_sp2tw.convert(query)   # 繁簡轉換
    query = query.replace("台", "臺").lower().strip()

    noise_words = ["請問", "一下", "呢", "嗎", "？", "呢？", "呢嗎", "請"]
    for word in noise_words:
        query = query.replace(word, "")
    return query

In [ ]:
# 文本向量化
def encode_texts(texts, model, tokenizer, device, pooling="cls", normalize_vec=True, batch_size=64):
    if isinstance(texts, str):
        texts = [texts]

    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encodings = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=64).to(device)
        with torch.no_grad():
            outputs = model(**encodings)
        if pooling == "cls":
            embeddings = outputs.last_hidden_state[:, 0]
        else:
            mask = encodings["attention_mask"].unsqueeze(-1).expand(outputs.last_hidden_state.size()).float()
            embeddings = (outputs.last_hidden_state * mask).sum(1) / mask.sum(1).clamp(min=1e-9)
        all_embeddings.append(embeddings.cpu())

    all_embeddings = torch.cat(all_embeddings, dim=0).numpy()
    return normalize(all_embeddings) if normalize_vec else all_embeddings

In [ ]:
# 查詢向量庫
def query_custom_sememe(query, index, metas, model, tokenizer, device, topk=10, pooling="cls", min_score=0.3):
    clean_query = preprocess_query(query)
    query_vec = encode_texts(clean_query, model, tokenizer, device, pooling)

    D, I = index.search(query_vec, topk)

    results = []
    for score, idx in zip(D[0], I[0]):
        if idx >= 0 and score >= min_score:
            meta = metas[idx]
            text = meta.get("term") or meta.get("query") or ""  # ⚡ 重點！安全 fallback
            results.append({
                "score": float(score),
                "text": text,
                "sememe": meta.get("sememe", ""),
                "is_location": meta.get("is_location", False)
            })
    return results

In [ ]:
# 地名簡化映射表
def build_city_alias_map(metas):
    alias_map = {}
    for meta in metas:
        if meta.get("is_location"):
            original = meta.get("term", "")
            if original:
                alias_map[original.replace("台", "臺")] = original
                short = original.replace("臺", "台").replace("市", "").replace("縣", "")
                alias_map[short] = original
    return alias_map

# term-sememe對應表
def build_term_sememe_map(metas):
    term_map = {}
    for meta in metas:
        term = meta.get("term", "")
        sememe = meta.get("sememe", "")
        if term and sememe:
            term_map[term] = sememe
    return term_map

# 地名fallback修正
def fix_location_name(name, city_alias_map):
    for key, val in city_alias_map.items():
        if key in name:
            return val
    return name

In [ ]:
# 天氣API查詢
def fetch_weather(location, api_keyW):
    try:
        url = f"http://api.openweathermap.org/data/2.5/weather?q={location}&appid={api_keyW}&units=metric&lang=zh_tw"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        return {
            "condition": data["weather"][0]["description"],
            "temperature": data["main"]["temp"],
            "humidity": data["main"]["humidity"],
            "pressure": data["main"]["pressure"],
            "wind_speed": data["wind"]["speed"]
        }
    except Exception as e:
        print(f"天氣API錯誤: {e} | 位置：{location}")
        return None

# 格式化回覆
def format_weather_reply(weather_data, location):
    if not weather_data:
        return f"無法取得 {location} 的天氣資料。"
    return (
        f"{location} 天氣概況：\n"
        f"狀態：{weather_data['condition']}\n"
        f"氣溫：{weather_data['temperature']}°C\n"
        f"濕度：{weather_data['humidity']}%\n"
        f"風速：{weather_data['wind_speed']} m/s\n"
        f"氣壓：{weather_data['pressure']} hPa"
    )

In [ ]:
# 向量檢索 + 地名修正 + fallback回覆
def handle_weather_query(user_query, index, metas, model, tokenizer, device, city_alias_map, term_sememe_map, api_keyW, topk=10, min_score=0.3):
    results = query_custom_sememe(user_query, index, metas, model, tokenizer, device, topk=topk, min_score=min_score)

    found_locations = []
    found_weather = False

    vague_terms = {"城市", "市區", "都市", "都會區", "市鎮"}

    for r in results:
        if r["is_location"] and r["text"] not in vague_terms:
            fixed_location = fix_location_name(r["text"], city_alias_map)
            found_locations.append(fixed_location)
        elif not r["is_location"] and ("天氣" in r["sememe"] or "氣候" in r["sememe"]):
            found_weather = True

    found_locations = list(set(found_locations))

    if found_locations and found_weather:
        replies = []
        for loc in found_locations:
            weather_data = fetch_weather(loc, api_keyW)
            reply = format_weather_reply(weather_data, loc)
            replies.append(reply)
        return "\n\n".join(replies)

    elif found_weather and not found_locations:
        return "請問您想查詢哪個城市或地區的天氣呢？"

    elif found_locations and not found_weather:
        replies = []
        for loc in found_locations:
            weather_data = fetch_weather(loc, api_keyW)
            reply = format_weather_reply(weather_data, loc)
            replies.append(reply)
        return "\n\n".join(replies)

    else:
        return "目前無法辨識您的問題是否與天氣有關，請重新描述地點與內容。"

In [ ]:
# 載入 custom index
index_custom = faiss.read_index("/content/custom_index.faiss")
with open("/content/custom_metadata.jsonl", "r", encoding="utf-8") as f:
    meta_custom = [json.loads(line) for line in f]

# 製作 alias_map 和 sememe_map
city_alias_map = build_city_alias_map(meta_custom)
term_sememe_map = build_term_sememe_map(meta_custom)

# 初始化 tokenizer + model
model_name = "BAAI/bge-base-zh"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to("cuda" if torch.cuda.is_available() else "cpu")
device = next(model.parameters()).device

In [ ]:
# 測試簡版
query = "今天台中會下雨嗎？"
print(preprocess_query(query))  # --> 確認有臺中和下雨字眼
results = query_custom_sememe(query, index_custom, meta_custom, model, tokenizer, device, topk=10, min_score=0.2)

for r in results:
    print(r)

In [ ]:
# 測試
user_query = "今天台中會下雨嗎？"
final_answer = handle_weather_query(user_query, index_custom, meta_custom, model, tokenizer, device, city_alias_map, term_sememe_map, api_keyW)
print(final_answer)

In [ ]:
# 調用 Groq API，根據檢索結果生成強化回答
def generate_answer_with_groq(user_input, results, api_key, model="mixtral-8x7b-32768", temperature=0.2):
    context_block = ""
    for idx, r in enumerate(results):
        query = r['meta'].get('query', '')
        sememe = r['meta'].get('sememe', '')
        context_block += f"[{idx+1}] {query} | {sememe}\n"

    prompt = (
        f"請根據以下已知資訊，回答用戶的問題。\n\n"
        f"===已知知識===\n{context_block}\n\n"
        f"===用戶提問===\n{user_input}\n\n"
        f"請保持簡潔、清楚，並避免編造。"
    )

    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": "你是一位專業的中文問答助手。"},
            {"role": "user", "content": prompt}
        ],
        "temperature": temperature,
        "top_p": 1.0,
        "max_tokens": 1024,
        "n": 1
    }

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=15)
        response.raise_for_status()
        data = response.json()
        return data["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"Groq API失敗: {e}")
        return "目前無法取得回答，請稍後再試。"

In [ ]:
def master_pipeline(user_input):
    if is_weather_query(user_input):
        return handle_weather_query(
            user_query=user_input,
            index=index_custom,
            metas=meta_custom,
            model=model_vector,
            tokenizer=tokenizer_vector,
            device=device_vector,
            city_alias_map=city_alias_map,
            term_sememe_map=term_sememe_map,
            api_keyW=api_keyW,
            topk=10,
            min_score=0.3
        )
    else:
        results = combine_search(
            query=user_input,
            indices=[index_nlpcc, index_custom],
            metadatas=[meta_nlpcc, meta_custom],
            model_vector=model_vector,
            tokenizer_vector=tokenizer_vector,
            device_vector=device_vector,
            topk=5
        )
        return generate_answer_with_groq(user_input, results, api_key, model=groq_model_id)

In [ ]:
# 載入向量庫與metadata
index_nlpcc, meta_nlpcc = vu.load_index_and_metadata("/content/index.faiss", "/content/metadata.jsonl")
index_custom, meta_custom = vu.load_index_and_metadata("/content/custom_index.faiss", "/content/custom_metadata.jsonl")

# 載入天氣向量庫
index_weather, meta_weather = vu.load_index_and_metadata("/content/weather_sememe_index.faiss", "/content/weather_sememe_metadata.jsonl")

# 載入知識庫用向量化模型
model_vector_name = "BAAI/bge-base-zh"
tokenizer_vector = AutoTokenizer.from_pretrained(model_vector_name)
model_vector = AutoModel.from_pretrained(model_vector_name).to("cuda" if torch.cuda.is_available() else "cpu")
device_vector = next(model_vector.parameters()).device

In [ ]:
# 測試
user_question = "今天台中會下雨嗎？"
final_answer = master_pipeline(user_question)
print(final_answer)

In [ ]:
# Chat 主邏輯
def chat_with_agent(user_input, history):
    history = history or []
    reply = master_pipeline(user_input)
    history.append((user_input, reply))
    return history, history

# 啟動 Gradio
with gr.Blocks(theme=gr.themes.Base(primary_hue="cyan")) as demo:
    gr.Markdown("""
    # 中文語義問答助理 + 天氣小幫手
     支援即時天氣查詢
     支援語義擴展推理
     RAG + LLM 智能生成
    """)

    with gr.Row():
        with gr.Column(scale=1):
            gr.Image(value="https://raw.githubusercontent.com/your_repo/your_logo.png", height=100)  # Logo（可以換網址或放本地）
        with gr.Column(scale=5):
            chatbot = gr.Chatbot(height=500, label="天氣型 AI 智能助理")

    with gr.Row():
        with gr.Column(scale=4):
            user_input = gr.Textbox(
                placeholder="請輸入問題，例如：明天高雄會下雨嗎？",
                show_label=False
            )
        with gr.Column(scale=1, min_width=80):
            submit_btn = gr.Button("送出", variant="primary")

    clear_btn = gr.Button("🧹 清除對話", variant="secondary")

    # 綁定交互
    history_state = gr.State([])

    submit_btn.click(
        chat_with_agent,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    user_input.submit(
        chat_with_agent,
        inputs=[user_input, history_state],
        outputs=[chatbot, history_state]
    )
    clear_btn.click(lambda: ([], []), inputs=None, outputs=[chatbot, history_state])

# 啟動服務
demo.launch(share=True)

Exception: Failed to download file. Status code: 400